# 2. PreProcessing

In this notebook, we're going to load the original csv file already described in the Introduction notebook, and look for data to clean or fix.
Finally, we'll save a new csv file, containing the preprocessed data, to be used in the next steps.

Let's import the pandas library to work on csv files and dataframes:

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("csv/df_atp.csv", engine='python')

Let's take a look at the data

In [3]:
df.head()

,Unnamed: 0,ATP,AvgL,AvgW,B&WL,B&WW,B365L,B365W,Best of,CBL,...,UBW,W1,W2,W3,W4,W5,WPts,WRank,Winner,Wsets
0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,6.0,6.0,NaN,NaN,NaN,NaN,63,Dosedel S.,2.0
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,6.0,6.0,NaN,NaN,NaN,NaN,5,Enqvist T.,2.0
2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,6.0,7.0,6.0,NaN,NaN,NaN,40,Escude N.,2.0
3,3,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,6.0,6.0,NaN,NaN,NaN,NaN,65,Federer R.,2.0
4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,7.0,5.0,6.0,NaN,NaN,NaN,81,Fromberg R.,2.0


Since we're not going to use every column in the dataset, first we're going to list all the columns, and then we'll delete the unnecessary ones.

In [4]:
list(df.columns)

['Unnamed: 0',
 'ATP',
 'AvgL',
 'AvgW',
 'B&WL',
 'B&WW',
 'B365L',
 'B365W',
 'Best of',
 'CBL',
 'CBW',
 'Comment',
 'Court',
 'Date',
 'EXL',
 'EXW',
 'GBL',
 'GBW',
 'IWL',
 'IWW',
 'L1',
 'L2',
 'L3',
 'L4',
 'L5',
 'LBL',
 'LBW',
 'LPts',
 'LRank',
 'Location',
 'Loser',
 'Lsets',
 'MaxL',
 'MaxW',
 'PSL',
 'PSW',
 'Round',
 'SBL',
 'SBW',
 'SJL',
 'SJW',
 'Series',
 'Surface',
 'Tournament',
 'UBL',
 'UBW',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'WPts',
 'WRank',
 'Winner',
 'Wsets']

In [5]:
df = df.drop(['Unnamed: 0','ATP','Tournament', 'Location', 'Best of', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 
              'Lsets', 'Comment','CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
              'B365W', 'B365L', 'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL', 'WPts',
              'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL'], axis=1)

In [6]:
df.head()

,AvgL,AvgW,Court,Date,LRank,Loser,MaxL,MaxW,Round,Series,Surface,WRank,Winner
0,NaN,NaN,Outdoor,2000-01-03,77,Ljubicic I.,NaN,NaN,1st Round,International,Hard,63,Dosedel S.
1,NaN,NaN,Outdoor,2000-01-03,56,Clement A.,NaN,NaN,1st Round,International,Hard,5,Enqvist T.
2,NaN,NaN,Outdoor,2000-01-03,655,Baccanello P.,NaN,NaN,1st Round,International,Hard,40,Escude N.
3,NaN,NaN,Outdoor,2000-01-03,87,Knippschild J.,NaN,NaN,1st Round,International,Hard,65,Federer R.
4,NaN,NaN,Outdoor,2000-01-03,198,Woodbridge T.,NaN,NaN,1st Round,International,Hard,81,Fromberg R.


Now, as we saw in the Introduction notebook, the 'Date' attribute is not a date, but it has 'object' type.
Let's convert these objects into dates:

In [7]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df['Date']

0       2000-01-03
1       2000-01-03
2       2000-01-03
3       2000-01-03
4       2000-01-03
           ...    
54903   2019-11-15
54904   2019-11-15
54905   2019-11-16
54906   2019-11-16
54907   2019-11-17
Name: Date, Length: 54908, dtype: datetime64[ns]

Now the 'Date' column contains dates only.

Next, if we take a look at 'Series' column, we can see that it contains old-labeled tournament series (i.e. International Gold for ATP500, etc.)

In [9]:
df['Series'].unique()

array(['International', 'Grand Slam', 'International Gold', 'Masters',
       'Masters Cup', 'ATP250', 'ATP500', 'Masters 1000'], dtype=object)

So, let's rename those values to make sure that each torunament category is different from the others.

In [10]:
df['Series'] = df['Series'].str.replace('International$','ATP250', regex=True)
df['Series'] = df['Series'].str.replace('International Series$','ATP250', regex=True)
df['Series'] = df['Series'].str.replace('International Gold$','ATP500', regex=True)
df['Series'] = df['Series'].str.replace('Masters$','Masters 1000', regex=True)

In [11]:
df['Series'].unique()

array(['ATP250', 'Grand Slam', 'ATP500', 'Masters 1000', 'Masters Cup'],
      dtype=object)

Now let's explore the 'WRank' and 'LRank' attributes:

In [12]:
#let's take the union of the sets of 'WRank' and 'LRank' values and visualize them
set(df["WRank"].unique()) | set(df['LRank'].unique())

{nan,
 '254',
 '641',
 '467',
 '236',
 '586.0',
 '17',
 '256.0',
 '1116.0',
 '5',
 '519',
 '29.0',
 '1333.0',
 '183',
 '277',
 '165.0',
 '434.0',
 '500',
 '113.0',
 '325.0',
 '430',
 '484',
 '502',
 '10.0',
 '192',
 '442',
 '352',
 '388',
 '771.0',
 '472',
 '920.0',
 '837.0',
 '459',
 '856',
 '457.0',
 '365',
 '748',
 '437',
 '127',
 '33',
 '207',
 '513.0',
 '838.0',
 '181.0',
 '44',
 '488.0',
 '383',
 '375',
 '289.0',
 '420',
 '478',
 '156.0',
 '740.0',
 '323',
 '868.0',
 '152',
 '114',
 '267.0',
 '352.0',
 '762.0',
 '728',
 '624.0',
 '284',
 '628.0',
 '1502.0',
 '19',
 '599',
 '220.0',
 '995.0',
 '555.0',
 '461.0',
 '4.0',
 '361',
 '174',
 '1497.0',
 '218',
 '655.0',
 '232',
 '88',
 '48.0',
 '567.0',
 '616.0',
 '275',
 '613',
 '186.0',
 '1059.0',
 '740',
 '349',
 '101',
 '715.0',
 '107.0',
 '80',
 '820.0',
 '231.0',
 '299.0',
 '1082.0',
 '1370.0',
 '1512.0',
 '164',
 '84',
 '41.0',
 '79.0',
 '1121.0',
 '269',
 '278.0',
 '529.0',
 '417',
 '67',
 '86',
 '272.0',
 '425',
 '854',
 '253',

As we can see, the values above are not numbers, but objects, and there are values like 'NR' or 'nan' (look at the end).

Let's change the types of columns WRank and LRank from object to int.
In order to do this, we firstly change their type to 'str', we replace values like 'NR' or 'nan' with the standard value of 500, and finally we cast the types to int.

In [13]:
df["WRank"] = df["WRank"].astype('str')
df["LRank"] = df["LRank"].astype('str')

df["LRank"] = df["LRank"].str.replace("NR$","500",regex=True)
df["WRank"] = df["WRank"].str.replace("NR$","500",regex=True)
df["LRank"] = df["LRank"].str.replace("nan$","500",regex=True)
df["WRank"] = df["WRank"].str.replace("nan$","500",regex=True)
df["WRank"] = df["WRank"].str.replace("\\.0$","",regex=True)
df["LRank"] = df["LRank"].str.replace("\\.0$","",regex=True)

df['WRank'] = df['WRank'].astype('int')
df['LRank'] = df['LRank'].astype('int')

In [14]:
df.head()

,AvgL,AvgW,Court,Date,LRank,Loser,MaxL,MaxW,Round,Series,Surface,WRank,Winner
0,NaN,NaN,Outdoor,2000-01-03,77,Ljubicic I.,NaN,NaN,1st Round,ATP250,Hard,63,Dosedel S.
1,NaN,NaN,Outdoor,2000-01-03,56,Clement A.,NaN,NaN,1st Round,ATP250,Hard,5,Enqvist T.
2,NaN,NaN,Outdoor,2000-01-03,655,Baccanello P.,NaN,NaN,1st Round,ATP250,Hard,40,Escude N.
3,NaN,NaN,Outdoor,2000-01-03,87,Knippschild J.,NaN,NaN,1st Round,ATP250,Hard,65,Federer R.
4,NaN,NaN,Outdoor,2000-01-03,198,Woodbridge T.,NaN,NaN,1st Round,ATP250,Hard,81,Fromberg R.


Another problem with this dataset, as we stated in the Introduction notebook, is that the players are already classified as "Winner" and "Loser".
Since we will train a model to identify who's more likely to win given a list of match attributes, we want to fix this issue, otherwise the model will figure out that columns relative to the winner are more 'important' than others to detect who'll win the match.

That's why we decided to:
- change the name of the columns "Winner" and "Loser" in "Player 0" and "Player 1".
- for half of the records, we'll swap "Player 1" attributes with "Player 0" attributes and viceversa, in this way we'll distribute the winners equally between "Player 0" and "Player 1". 
- create a new column "Won" that will contain 0 if Player 0 has won the match or 1 otherwise.

In [15]:
df = df.rename({"Winner":"Player 0", "Loser":"Player 1", 
                "WRank":"Pl0_Rank", "LRank":"Pl1_Rank",
                "MaxW":"Max_Pl0", "MaxL":"Max_Pl1", "AvgW":"Avg_Pl0", "AvgL":"Avg_Pl1"
               }, axis=1)

In [16]:
for ix, row in df.iterrows():
    if ix % 2 == 0: 
        #swap half of the records
        old_p0 = row['Player 0']
        old_p1 = row['Player 1']
        old_p0_rank = row['Pl0_Rank']
        old_p1_rank = row['Pl1_Rank']
        old_max_p0 = row['Max_Pl0']
        old_max_p1 = row['Max_Pl1']
        old_avg_p0 = row['Avg_Pl0']
        old_avg_p1 = row['Avg_Pl1']

        df.at[ix,'Player 0'] = old_p1
        df.at[ix,'Player 1'] = old_p0
        df.at[ix,'Pl0_Rank'] = old_p1_rank
        df.at[ix,'Pl1_Rank'] = old_p0_rank
        df.at[ix,'Max_Pl0'] = old_max_p1
        df.at[ix,'Max_Pl1'] = old_max_p0
        df.at[ix,'Avg_Pl0'] = old_avg_p1
        df.at[ix,'Avg_Pl1'] = old_avg_p0

        df.at[ix,'Won'] = 1
    else:
        #don't swap these records, but just mark as winner Player 0
        df.at[ix,'Won'] = 0

In [17]:
df.head()

,Avg_Pl1,Avg_Pl0,Court,Date,Pl1_Rank,Player 1,Max_Pl1,Max_Pl0,Round,Series,Surface,Pl0_Rank,Player 0,Won
0,NaN,NaN,Outdoor,2000-01-03,63,Dosedel S.,NaN,NaN,1st Round,ATP250,Hard,77,Ljubicic I.,1.0
1,NaN,NaN,Outdoor,2000-01-03,56,Clement A.,NaN,NaN,1st Round,ATP250,Hard,5,Enqvist T.,0.0
2,NaN,NaN,Outdoor,2000-01-03,40,Escude N.,NaN,NaN,1st Round,ATP250,Hard,655,Baccanello P.,1.0
3,NaN,NaN,Outdoor,2000-01-03,87,Knippschild J.,NaN,NaN,1st Round,ATP250,Hard,65,Federer R.,0.0
4,NaN,NaN,Outdoor,2000-01-03,81,Fromberg R.,NaN,NaN,1st Round,ATP250,Hard,198,Woodbridge T.,1.0


Next, we are going to create dummy variables dataframe for the columns: 'Court', 'Surface', 'Round' and 'Series'.

Then we are going to add these dummy variables dataframes to the original one.

In [18]:
court_dummies = pd.get_dummies(df['Court'])
court_dummies.head()

,Indoor,Outdoor
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [19]:
surface_dummies = pd.get_dummies(df['Surface'])
surface_dummies.head()

,Carpet,Clay,Grass,Hard
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [20]:
round_dummies = pd.get_dummies(df['Round'])
round_dummies.head()

,1st Round,2nd Round,3rd Round,4th Round,Quarterfinals,Round Robin,Semifinals,The Final
0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


In [21]:
series_dummies = pd.get_dummies(df['Series'])
series_dummies.head()

,ATP250,ATP500,Grand Slam,Masters 1000,Masters Cup
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


Now we're going to merge all these dataframes and append those to the original one:

In [22]:
df = pd.concat([df, court_dummies, surface_dummies, series_dummies, round_dummies], axis=1)
df.head()

,Avg_Pl1,Avg_Pl0,Court,Date,Pl1_Rank,Player 1,Max_Pl1,Max_Pl0,Round,Series,...,Masters 1000,Masters Cup,1st Round,2nd Round,3rd Round,4th Round,Quarterfinals,Round Robin,Semifinals,The Final
0,NaN,NaN,Outdoor,2000-01-03,63,Dosedel S.,NaN,NaN,1st Round,ATP250,...,0,0,1,0,0,0,0,0,0,0
1,NaN,NaN,Outdoor,2000-01-03,56,Clement A.,NaN,NaN,1st Round,ATP250,...,0,0,1,0,0,0,0,0,0,0
2,NaN,NaN,Outdoor,2000-01-03,40,Escude N.,NaN,NaN,1st Round,ATP250,...,0,0,1,0,0,0,0,0,0,0
3,NaN,NaN,Outdoor,2000-01-03,87,Knippschild J.,NaN,NaN,1st Round,ATP250,...,0,0,1,0,0,0,0,0,0,0
4,NaN,NaN,Outdoor,2000-01-03,81,Fromberg R.,NaN,NaN,1st Round,ATP250,...,0,0,1,0,0,0,0,0,0,0


Now we're going to the delete the 'Court', 'Surface', 'Round' and 'Series' columns since we already created the respective dummy columns:

In [23]:
df.drop(['Court', 'Surface', 'Round', 'Series'], axis=1, inplace = True)

In [24]:
df.head()

,Avg_Pl1,Avg_Pl0,Date,Pl1_Rank,Player 1,Max_Pl1,Max_Pl0,Pl0_Rank,Player 0,Won,...,Masters 1000,Masters Cup,1st Round,2nd Round,3rd Round,4th Round,Quarterfinals,Round Robin,Semifinals,The Final
0,NaN,NaN,2000-01-03,63,Dosedel S.,NaN,NaN,77,Ljubicic I.,1.0,...,0,0,1,0,0,0,0,0,0,0
1,NaN,NaN,2000-01-03,56,Clement A.,NaN,NaN,5,Enqvist T.,0.0,...,0,0,1,0,0,0,0,0,0,0
2,NaN,NaN,2000-01-03,40,Escude N.,NaN,NaN,655,Baccanello P.,1.0,...,0,0,1,0,0,0,0,0,0,0
3,NaN,NaN,2000-01-03,87,Knippschild J.,NaN,NaN,65,Federer R.,0.0,...,0,0,1,0,0,0,0,0,0,0
4,NaN,NaN,2000-01-03,81,Fromberg R.,NaN,NaN,198,Woodbridge T.,1.0,...,0,0,1,0,0,0,0,0,0,0


Finally, the preprocessing stage is concluded!

Now, we're going to save the preprocessed dataframe in a csv file, so that the next notebook will use this dataframe directly after loading it!

In [25]:
df.to_csv("csv/Preprocessed_Data.csv", index=False)